In [1]:
import sys
import os
sys.path.append('/data/GitHub/lca-tools/')

import lcatools

In [2]:
from lcatools.providers.ecospold2 import EcospoldV2Archive
from lcatools.entities import NoReferenceFound

In [3]:
sys.path.append('..')
import lcamatrix

In [4]:
CATALOG = '/data/GitHub/lca-catalog/catalogs/'

EI32_A = 'ecoinvent_3.2_apos_spold.json.gz'
#USLCI = 'uslci_ecospold.json.gz'

In [134]:
EA = EcospoldV2Archive('/data/LCI/Ecoinvent/3.2/current_Version_3.2_apos_ecoSpold02.zip', prefix='datasets')

Found Extension: zip


In [135]:
EA.load_all()

Loading /data/LCI/Ecoinvent/3.2/current_Version_3.2_apos_ecoSpold02.zip
 Loaded 100 processes (t=0.96 s)
 Loaded 200 processes (t=1.78 s)
 Loaded 300 processes (t=2.37 s)
 Loaded 400 processes (t=2.90 s)
 Loaded 500 processes (t=3.29 s)
activity ID 0a3fc02d-87cb-4200-88ac-ce5fc0ad5f42: no comment
 Loaded 600 processes (t=4.01 s)
 Loaded 700 processes (t=4.57 s)
 Loaded 800 processes (t=5.01 s)
 Loaded 900 processes (t=5.82 s)
 Loaded 1000 processes (t=6.37 s)
 Loaded 1100 processes (t=6.82 s)
 Loaded 1200 processes (t=7.49 s)
 Loaded 1300 processes (t=8.35 s)
 Loaded 1400 processes (t=8.95 s)
 Loaded 1500 processes (t=9.43 s)
 Loaded 1600 processes (t=10.23 s)
 Loaded 1700 processes (t=10.78 s)
 Loaded 1800 processes (t=11.22 s)
 Loaded 1900 processes (t=11.96 s)
 Loaded 2000 processes (t=13.06 s)
 Loaded 2100 processes (t=13.62 s)
 Loaded 2200 processes (t=14.45 s)
 Loaded 2300 processes (t=15.21 s)
 Loaded 2400 processes (t=15.67 s)
 Loaded 2500 processes (t=16.26 s)
 Loaded 2600 pro

In [8]:
%time EA.write_to_file(os.path.join(CATALOG, EI32_A), exchanges=True, values=True, characterizations=True, gzip=True)

CPU times: user 28.3 s, sys: 200 ms, total: 28.5 s
Wall time: 28.8 s


In [5]:
%time EA_j = lcatools.archive_from_json(os.path.join(CATALOG, EI32_A))

Loading JSON data from /data/GitHub/lca-catalog/catalogs/ecoinvent_3.2_apos_spold.json.gz:
Found Extension: zip
11420 new process entities added (11420 total)
4650 new flow entities added (4650 total)
19 new quantity entities added (19 total)
CPU times: user 13.5 s, sys: 297 ms, total: 13.8 s
Wall time: 13.8 s


In [5]:
import cProfile

In [6]:
cProfile.run('EA_j = lcatools.archive_from_json(os.path.join(CATALOG, EI32_A))')

Loading JSON data from /data/GitHub/lca-catalog/catalogs/ecoinvent_3.2_apos_spold.json.gz:
Found Extension: zip
11420 new process entities added (11420 total)
4650 new flow entities added (4650 total)
19 new quantity entities added (19 total)
         26159639 function calls (26139571 primitive calls) in 17.902 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:102(release)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:142(__init__)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:146(__enter__)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:153(__exit__)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:159(_get_module_lock)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:173(cb)
        1    0.000    0.000    0.0

## Validate json allocated exchange integrity
Someday I will actually start to run test-driven development. but not today.


In [136]:
def compare_all_exchanges(defender, contender):
    failed = False
    for x in defender.exchanges():
        xj = contender._exchanges[x.key]
        # if x in defender.reference_entity:
        if x.value != xj.value:
            if failed is False:
                failed = True
                defender.show()
            print('Fail on reference %s' % x)
        if x._value_dict != xj._value_dict:
            if failed is False:
                failed = True
                defender.show()
            print('fail on %s' % x)
    return not failed

In [137]:
for p in EA.processes():
    failed = []
    json_p = EA_j[p.uuid]
    if compare_all_exchanges(p, json_p) is False:
        failed.append(p)


In [138]:
failed

[]

In [139]:
len(EA.processes())

11420

In [140]:
ps = EA.search(entity_type='process', Name='yogurt')

In [141]:
str(ps[0]) == 'yogurt production, from cow milk [CA-QC]'

True

In [142]:
p = ps[0]

In [143]:
p.get_uuid() == '580dc5e1-8866-4caa-848e-456476fde542'

True

In [144]:
for x in p.reference_entity:
    print(str(x))

yogurt production, from cow milk [CA-QC] has Output: cheese, from cow milk, fresh, unripened [By-product classification: allocatable product] [kg]
yogurt production, from cow milk [CA-QC] has Output: cream, from cow milk [By-product classification: allocatable product] [kg]
yogurt production, from cow milk [CA-QC] has Output: yogurt, from cow milk [By-product classification: allocatable product] [kg]


In [145]:
yog = p.find_reference('yogurt')

In [146]:
yog

In [147]:
import cProfile

In [64]:
import importlib

In [119]:
importlib.reload(lcamatrix.background)

<module 'lcamatrix.background' from '../lcamatrix/background.py'>

In [120]:
from lcamatrix.background import BackgroundManager

In [148]:
B = BackgroundManager(EA)

In [151]:
## and it's even faster without the profiler ~~ 10 secs to add ALL product flows and construct matrices!
#14.801 - 4.481 - 2.799(1.430) - 0.769 -0.344 - 0.599 - 0.67 - 0.560 - 0.360  ### process _exchanges key; ditch from_allocated; other extensive optimizations
## ^^major work exchanges--> exchanges.__getitem__
#22.832 -11.135 - 3.813(1.979) - 2.279(0.369) - 1.159 - 0.92 - 0.859 - 0.404  ### take out sorting from process.exchanges(); take out unit from exchange.__init__
#22.558 -11.126 - 3.622(1.884) - 2.437(0.358) - 1.128 - 0.91 - 0.781 - 0.378  ### finish out the static exchange hash- ffs
#26.739 -15.131 - 3.843(2.022) - 1.982(0.369) - 1.235 - 0.97 - 0.830 - 0.435  ### take 3 on try:except, with prior - better
#26.839 -14.975 - 4.122(2.014) - 2.005(0.366) - 1.234 - 0.97 - 0.817 - 0.437  ### uuid key AGAIN! in _terminations
#28.251 -15.266 - 4.520(2.107) - 2.086(0.395) - 1.278 - 1.38 - 0.869 - 0.446  ### nope nope nope no effect.
#28.561 -15.826 - 4.640()  ### revert try:except, make resolve_termination static class method
#27.160 -14.882 - 4.193(1.999) - 2.334(0.388) - 1.224 - 0.97 - 0.826 - 0.444  ### finish lowlink rework
#27.156 -14.465 - 4.151(2.002) - 1.967(0.348) - 1.229 - 1.89 - 0.803 - 0.422  ### static exchange hash -- full reload
#32.509 -19.952 - 4.197(2.004) - 2.021(0.378) - 1.165 - xxxx - 0.835 - 0.461  ### _lowlink and index key use
#32.909 -20.001 - 4.144(1.985) - 2.227(0.377) - 1.168 - 1.20 - 0.836 - 0.469  ### fix Emission._hash!!
#34.984 -19.904 - 4.300(2.027) - 3.845(2.160) - 1.184 - 1.53 - 0.848 - 0.457  ### get uuid.UUID out of to_uuid (except when required)-- full reload 
#37.140 -19.265 - 7.578(5.410) - 3.750(2.086) - 1.214 - 1.15 - 0.880 - 0.436  ### try:except KeyError in terminate
#36.769 -18.346 - 7.249(4.893) - 4.784(2.854) - 1.217 - 1.12 - 0.842 - 0.418  ### cache cutoffs in _terminations
#36.420 -19.178 - 6.907        - 4.038(2.402) - 1.173 - 1.13 - 0.837 - 0.428  ## pf / ef keys to tuples
#37.782 - 19.522 - 7.111 - 3.904 - 1.211 - 1.16 - 2.217 - 0.453 
# _traverse - exchanges - terminate(__getitem__) - add_cutoff(_add_emission) - add_interior - _set_lowlink - _check_product_flow - _create_product_flow
# leaves remainder:

In [149]:
cProfile.run('B.add_ref_product(yog.flow, p)')

         37914055 function calls (37903508 primitive calls) in 18.514 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:402(parent)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:989(_handle_fromlist)
        1    0.000    0.000   18.514   18.514 <string>:1(<module>)
   603227    0.216    0.000    0.478    0.000 <string>:12(__new__)
        4    0.000    0.000    0.001    0.000 _methods.py:25(_amax)
        4    0.000    0.000    0.001    0.000 _methods.py:28(_amin)
   251199    0.173    0.000    0.327    0.000 background.py:102(check_stack)
    10283    0.016    0.000    0.044    0.000 background.py:109(add_to_stack)
      263    0.014    0.000    0.034    0.000 background.py:117(label_scc)
        1    0.000    0.000    0.009    0.009 background.py:132(_set_background)
    262/1    0.000    0.000    0.000    0.000 backgr

In [121]:
Bj = BackgroundManager(EA_j)

In [122]:
%time Bj.add_all_ref_products()

CPU times: user 10.9 s, sys: 30 ms, total: 10.9 s
Wall time: 10.9 s


In [123]:
len(Bj._cutoff)

58500

In [124]:
len([b for b in Bj.tstack.background_flows()])

10282

In [125]:
sorted([len(scc) for k, scc in Bj.tstack._sccs.items()], reverse=True)[:30]

[10021,
 6,
 6,
 6,
 6,
 5,
 5,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [126]:
len(Bj.tstack._downstream)

261

In [127]:
len(Bj._product_flows)

12966

In [128]:
len(Bj._interior)

251170

In [129]:
Bj._a_matrix.nnz

251170

In [130]:
278775 - 251170

27605

In [131]:
len(Bj._foreground)

27605

In [132]:
fg = [f for f in Bj.tstack.foreground_flows()]

In [133]:
len(fg)

2654

In [72]:
fgo = [f for f in Bj.tstack.foreground_flows(outputs=True)]
len(fgo)

958

In [89]:
%time fg_size = [len(Bj.tstack.foreground(k)) for k in fgo]

CPU times: user 6.67 ms, sys: 0 ns, total: 6.67 ms
Wall time: 5 ms


In [90]:
max(fg_size)

43

In [91]:
fg_size.index(max(fg_size))

735

In [92]:
fg_tree = Bj.tstack.foreground(fgo[735])

In [93]:
[Bj.product_flow(k).process['Name'] for k in fg_tree]

['market for operation, computer, desktop, home use',
 'operation, computer, desktop, with liquid crystal display, home use',
 'operation, computer, desktop, with liquid crystal display, home use',
 'operation, computer, desktop, with cathode ray tube display, home use',
 'operation, computer, desktop, with liquid crystal display, home use',
 'operation, computer, desktop, with cathode ray tube display, home use',
 'operation, computer, desktop, with cathode ray tube display, home use',
 'operation, computer, desktop, with cathode ray tube display, home use',
 'operation, computer, desktop, with liquid crystal display, home use',
 'operation, computer, desktop, with liquid crystal display, active mode',
 'operation, computer, desktop, with liquid crystal display, off mode',
 'operation, computer, desktop, with liquid crystal display, standby mode',
 'market for operation, computer, desktop, with liquid crystal display, active mode',
 'market for operation, computer, desktop, with liqui

In [42]:
len(Bj.tstack.foreground(fgo[63]))

KeyboardInterrupt: 

In [45]:
from time import sleep

In [60]:
    def foreground(self, index):
        """
        returns a list of foreground SCCs that are downstream of the supplied scc ID (itself included)
        This is a list of components necessary to build the foreground fragment tree
        :param index:
        :return: topologically-ordered, loop-detecting list of non-background SCC IDs
        """
        if self.is_background(index):
            return []
        queue = [index]
        fg = []
        while len(queue) > 0:
            print('  nq: %s' % queue)
            current = queue.pop(0)
            print('c: %d' % current)
            sleep(0.020)
            if current not in fg:
                queue.extend([k for k in self._component_rows_by_col[current] if not self.is_background(k)])
                print('   q: %s' % queue)
                sleep(0.050)
                fg.append(current)
            else:
                print('     loop')
        return fg


In [61]:
foreground(Bj.tstack, fgo[63])

  nq: [10748]
c: 10748
   q: [10749, 10647]
  nq: [10749, 10647]
c: 10749
   q: [10647, 10648]
  nq: [10647, 10648]
c: 10647
   q: [10648, 10648]
  nq: [10648, 10648]
c: 10648
   q: [10648, 10648]
  nq: [10648, 10648]
c: 10648
     loop
  nq: [10648]
c: 10648
     loop


[10748, 10749, 10647, 10648]

In [32]:
Bj.tstack.foreground(fgo[8])

[10340, 10344, 10341, 10284, 10342, 10284, 10342, 10285, 10343, 10285, 10343]

In [45]:
len(Bj.tstack._sccs)

2840

In [46]:
2840-2654

186

In [48]:
10097 + 185

10282

In [66]:
len(Bj._product_flows)

10283

In [57]:
bgen = (b for b in Bj.tstack.background_flows())

In [59]:
pf = next(bgen)

In [63]:
str(pf.flow)

'electricity, high voltage [By-product classification: allocatable product]'

In [53]:
len(Bj._emissions)

1937

In [50]:
%time [str(x) for x in Bj._terminations[(pf.key[0], 'Input')]]

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 21 µs


['yogurt production, from cow milk [CA-QC]',
 'market for yogurt, from cow milk [GLO]',
 'yogurt production, from cow milk [RoW]']

In [44]:
str(EA_j[pf.key[1]])

'yogurt production, from cow milk [CA-QC]'

In [75]:
len(B._product_flows)

10283

In [54]:
%time bg = [b.index for b in B.tstack.background_flows()]

CPU times: user 3.33 ms, sys: 3.33 ms, total: 6.67 ms
Wall time: 6.26 ms


In [55]:
%time bg_dict = dict((v, k) for k, v in enumerate(bg))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.07 ms


In [56]:
%time B._construct_b_matrix(bg_dict)

CPU times: user 690 ms, sys: 20 ms, total: 710 ms
Wall time: 708 ms


In [58]:
B._cutoff

[MatrixEntry(parent=<lcamatrix.foreground.ProductFlow object at 0x7fc807651278>, term=<lcamatrix.foreground.Emission object at 0x7fc80e21de80>, value=0.00391255795812271),
 MatrixEntry(parent=<lcamatrix.foreground.ProductFlow object at 0x7fc807651278>, term=<lcamatrix.foreground.Emission object at 0x7fc80e037208>, value=5.25921928345733e-09),
 MatrixEntry(parent=<lcamatrix.foreground.ProductFlow object at 0x7fc807651278>, term=<lcamatrix.foreground.Emission object at 0x7fc80e0cf7b8>, value=1.33893853637212e-07)]

In [60]:
.865 + .678 + 37.782

39.324999999999996

In [34]:
pd = EA['55ee5166-8fd9-4955-937c-69b0a66f30ee']

In [37]:
refs = list(pd.reference_entity)

In [49]:
pd.is_allocated(refs[0])

True

In [56]:
len(F._cutoff) + len(F._interior) == 791758

True

In [68]:
pf = list(F._product_flows.keys())

In [74]:
F.tstack.scc_id(pf[3336])

1

In [79]:
['%s: %s' % (pf[i]._process['Name'], pf[i]._flow['Name']) for i in (3337, 3338)]

['market for electricity, low voltage: electricity, low voltage',
 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted: electricity, low voltage']

In [80]:
from collections import defaultdict

In [81]:
d = defaultdict(set)

In [89]:
from collections import Counter

In [103]:
elem = {'air', 'water', 'soil', 'resource'}

output_counter = Counter()

for p in us['processes']:
    for x in p['exchanges']:
        if x['direction'] == 'Output':
            if x['flow'] in in_us:
            #if us_flows[x['flow']]['Compartment'][0] not in elem:
                output_counter[x['flow']] += 1
            

In [182]:
for k in output_counter.most_common(30):
    print('%2d: (%s) %s' % (k[1], us_flows[k[0]]['externalId'], us_flows[k[0]]['Name']))

86: (5853) Chromium
61: (6619) Chloride
54: (6621) Sodium, ion
50: (5875) Carbon dioxide, biogenic
19: (6615) Chromium, ion
16: (6601) Thallium
 8: (6605) Hydrogen cyanide (prussic acid)
 8: (6603) Chloride (unspecified)
 8: (7907) Inorganic salts and acids, unspecified
 8: (9688) Water, cooling
 7: (6617) Methane, bromotrifluoro-, Halon 1301
 7: (15462) Cooling water, non-contact
 3: (13968) Water
 2: (775) Diesel, at refinery
 2: (6403) Oil, crude, 42 MJ per kg, in ground
 2: (16700) Kerosene, at refinery
 2: (7295) Quartz sand (silica sand; silicon dioxide)
 2: (7293) Sodium chloride, in ground
 2: (16696) Residual fuel oil, at refinery
 2: (7352) Lignite, 11 MJ per kg, in ground
 2: (13406) Petroleum coke, at refinery
 2: (16694) Liquefied petroleum gas, at refinery
 2: (6903) Oxygen
 2: (6401) Gas, natural, 46.8 MJ per kg, in ground
 2: (7350) Coal, hard, 30.7 MJ per kg, in ground
 2: (23293) Harvesting, fresh fruit bunch, at farm
 2: (869) Gasoline, at refinery
 2: (5339) Wood fu